# Tools
#### TODO LIST
1. Asset statistics Table   
##### AcountDict

ID | AcountID | AcountCatalog | AcountName | AcountAmount | TargetAmount |ProjectNo1 | ProjectName
---|---|---|---|---|---|---|---
1|001|Finance|ABC-0815|1000|10000|01ade|Pro1


##### AcountTB
DateTime | AcountID | AcountName | AcountAmount | 
---|---|---|---

## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import baostock as bs
import math
import pickle

from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [17]:
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+ lg.error_code)
print('login respond  error_msg:'+ lg.error_msg)

#### 获取历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节
rs = bs.query_history_k_data_plus("sh.600000",
    "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST",
    start_date='2019-06-01', end_date='2019-12-31', 
    frequency="d", adjustflag="3") #frequency="d"取日k线，adjustflag="3"默认不复权
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
#### 结果集输出到csv文件 ####
# result.to_csv("D:/history_k_data.csv", encoding="gbk", index=False)
# print(result)
result.tail()

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success


,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST
140,2019-12-25,sh.600000,12.2900,12.2900,12.2000,12.2400,12.2800,13678175,167407896.0000,3,0.048700,1,-0.325700,5.884165,0.748224,1.880224,-5.959912,0
141,2019-12-26,sh.600000,12.2600,12.3200,12.2300,12.2900,12.2400,15739054,193083033.0000,3,0.056000,1,0.408500,5.908202,0.751280,1.887905,-5.984258,0
142,2019-12-27,sh.600000,12.2500,12.4300,12.2500,12.3200,12.2900,27932109,344832499.0000,3,0.099400,1,0.244100,5.922624,0.753114,1.892513,-5.998866,0
143,2019-12-30,sh.600000,12.2700,12.3600,12.1200,12.3400,12.3200,41051555,503090483.0000,3,0.146100,1,0.162300,5.932238,0.754337,1.895585,-6.008604,0
144,2019-12-31,sh.600000,12.3200,12.3800,12.2100,12.3700,12.3400,31953628,392736236.0000,3,0.113700,1,0.243100,5.946660,0.756171,1.900194,-6.023212,0


In [ ]:
#### 登出系统 ####
bs.logout()

## Function

In [31]:
# 函数
def xnpv(rate, cashflows):
    return sum([cf/(1+rate)**((t-cashflows[0][0]).days/365.0) for (t,cf) in cashflows])


def xirr(cashflows, guess=0.1):
    try:
        res = optimize.newton(lambda r: xnpv(r,cashflows),guess)
        return print('Yield: {:.2%}'.format(res))
    except:
        print('Calc Wrong')

In [37]:
# 函数
def xirr2(cashflows):
    """
    计算现金流计划的内部收益率
    """
    years = [(ta[0] - cashflows[0][0]).days / 365. for ta in cashflows]
    residual = 1.0
    step = 0.05
    guess = 0.05
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, trans in enumerate(cashflows):
            residual += trans[1] / pow(guess, years[i])
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return print("Yield:{:.2%}".format(guess - 1))

In [34]:
data = ['2017/1/20','2017/2/17','2017/3/20','2017/4/18','2017/5/15','2017/6/19','2017/7/18','2017/8/21','2017/9/18','2017/10/16','2017/11/20','2017/12/19','2017/12/19']
DATE = [datetime.datetime.strptime(i, '%Y/%m/%d') for i in data]
amount = [-749.7,-569,-582,-553.5,-681.6,-523.8,-789.5,-521.1,-580.2,-591.2,-676.1,-657.9,7931.5]
test =  [i for i in zip(DATE, amount)]

In [38]:
xirr2(test)

Yield:13.53%


In [11]:
import datetime

if 8 < int(datetime.datetime.strftime(datetime.datetime.now(), '%H')) < 21:
    sec = 10
else:
    sec = 120
print(sec)

10


In [20]:
from string import Template

file = r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test\db\sql\rtdm_decision_log_monitor_test.sql'

with open(file, 'r', encoding='utf8') as f:
    sql = Template(f.read()).substitute(sec=sec)
print(sql)

select COUNT(*) from app_rtcdm.cie_rtdm_decision_log lo where lo.event_dt + 10/(24*60*60) > sysdate


In [1]:
import configparser

config = configparser.ConfigParser()
config.read(r'C:\Users\leo.zhangzs\AppData\Roaming\Code\User\VScode\py3\UAT_Test\core\Setting.txt', encoding='utf-8')

['C:\\Users\\leo.zhangzs\\AppData\\Roaming\\Code\\User\\VScode\\py3\\UAT_Test\\core\\Setting.txt']

In [2]:
role = config.get('Configs','ROLE_LIST')

In [5]:
role_list = role.split(',')

In [7]:
("','").join(role_list)

"EMPLOYEE','TIPPER','SA','RA','DSM','DSMT','SDSM','EMP','CH"

In [2]:
res = pd.DataFrame()

In [1]:
import os

f1 = r'C:\Users\leo.zhangzs\Documents\HCC\CAPP Monitor Report\Use Case\01 AF STUCK\af stuck from pre_core.sql'

In [4]:
os.path.splitext(f1)[1]

'.sql'